# Lectura de los datos 

In [33]:

from math import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np

##Lectura de datos

In [34]:
data = pd.read_csv("./framingham.csv")

data = data.dropna()

data.isna().sum()



male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


## Manipulación, balanceo y exploración de datos

In [35]:

X = data.drop("TenYearCHD", axis=1)
y = data["TenYearCHD"]

oversampler = RandomOverSampler()

X_resampled, y_resampled = oversampler.fit_resample(X, y)

print(y_resampled.value_counts())

data = pd.concat([X_resampled, y_resampled], axis=1)


0    3099
1    3099
Name: TenYearCHD, dtype: int64


### Normalizacion y escalacion de los datos

In [36]:
columns_to_normalize = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'heartRate', 'glucose']

scaler = StandardScaler()

scaled_data = scaler.fit_transform(data[columns_to_normalize])
df_scaled = pd.DataFrame(scaled_data, columns=columns_to_normalize)
data = pd.concat([data.drop(columns_to_normalize, axis=1), df_scaled], axis=1)

data.head()


,male,age,education,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,heartRate,glucose
0,1,39,4.0,0,0.0,0,0,0,0,-0.782683,-0.973729,-1.270400,-1.112125,0.204792,0.342877,0.342877,-0.234107
1,0,46,2.0,0,0.0,0,0,0,0,-0.782683,0.190994,-0.653974,-0.272066,0.617849,1.601606,1.601606,-0.265360
2,1,48,1.0,1,0.0,0,0,0,0,0.841269,0.085110,-0.386856,-0.348435,-0.177755,-0.076699,-0.076699,-0.452880
3,0,61,3.0,1,0.0,0,1,0,1,1.653245,-0.338425,0.537783,0.797100,0.582646,-0.915851,-0.915851,0.578477
4,0,46,3.0,1,0.0,0,0,0,0,1.084861,0.932182,-0.284118,-0.042959,-0.703464,0.762453,0.762453,0.015919


### PCA

In [37]:

data = data.dropna()

y = data['TenYearCHD']
y = y.to_numpy()
X = data.drop('TenYearCHD', axis=1)

pca = PCA(n_components=7)
X = pca.fit_transform(X)

## División de data de entrenamiento y prueba

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=1234)

## Fit de la regresión logística polinomial

In [39]:

imputer = SimpleImputer(strategy='mean')

# Ajusta el imputer a los datos de entrenamiento y transforma los datos de entrenamiento
X_train_imputed = imputer.fit_transform(X_train)

# Usa el mismo imputer para transformar los datos de prueba
X_test_imputed = imputer.transform(X_test)

lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train_imputed, y_train)
y_pred = lr.predict(X_test_imputed)

confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)



0.6879032258064516

In [40]:

# Crear un pipeline con un transformador polinómico y un modelo de regresión lineal
pipeline = Pipeline([('polynomial_features', PolynomialFeatures()),
                     ('linear_regression', LinearRegression())])

# Especificar los grados del polinomio que desea probar
degrees = np.arange(1, 5)

# Especificar los parámetros de la búsqueda de cuadrícula
param_grid = {'polynomial_features__degree': degrees}

# Realizar la búsqueda de cuadrícula con validación cruzada de 5 particiones
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X, y)

# Obtener el grado del polinomio óptimo
best_degree = grid_search.best_params_['polynomial_features__degree']

# Imprimir el grado del polinomio óptimo
print('Grado del polinomio óptimo:', best_degree)

Grado del polinomio óptimo: 3


## Discusión de resultados

En primer lugar se pudo observar que el modelo sufre de overfiling cuando no se balancea porque casi llega a un acurracy de 0.9. Sin embargo, al balancear los datos, el modelo se vuelve más robusto y se puede observar que el acurracy mas o menos de 0.7. Esto se debe a que el modelo se vuelve más general y no se ajusta a los datos de entrenamiento.

Luego de ello al utilizar cross validation se pudo determinar que el grado de polinimio que mejor se ajusta a la data es el grado 3. Esto se debe a que el acurracy es mayor que en los demás grados de polinomio. 